# Comparative Study of Dynamic Programming and Genetic Algorithms for the Solution of 0/1 Knapsack Problem

### Author Ivan Georgiev

## Abstract 

The 0/1 Knapsack Problem, involves selecting a subset of items with given weights and values to maximize total value without exceeding a capacity limit. This project aims to compares two distinct approaches to solving this challange: Dynamic Programming (DP), which guarantees an optimal solution, and Genetic Algorithms (GA), a heuristic method inspired by natural evolution. By implementing both algorithms in Python, we evaluate their performance in terms of runtime and solution quality across varying problem sizes. Experiments reveal that DP excels in smaller instances with its precision, while GA offers scalability for larger instances at the cost of optimality. These findings highlight the trade-offs between exact and approximate methods, providing insights into their practical applicability in resource allocation and beyond.

## Introduction 

The 0/1 Knapsack Problem is a fundamental challenge within the field of combinatorial optimization. The problem is related to the following: in a given set of items, with a different weight and values, the task is to select the most valuable subset of these items that can be accommodated within a knapsack with a specific capacity. The stipulation is that each item can be either entirely excluded or included in the subset and that's what 0/1 means in the name of the challenge.   
The problem is very important in theory and practice and can find numerous implications including efficient budget allocations in budget constraints situations, optimization solutions in transportation, logistics and cargo, and even applications in the fields of projects planning and resources distribution.    
The well-known solution of the problem involves the Dynamic Programing(DP) algorithm. The algorithm's strengths are based in its systematic methodology related to the principles of optimal substructure and overlapping subproblems. It will always find the optimal solution but the pseudo-polynomial time complexity related to the DP algorithm $O(nW)$ (where $n$ is number of items and $W$ is the capacity) can lead to computational inefficiencies when the number of items or the knapsack capacity become too large. 
In that perspective, the challenge when the number of knapsack instances becomes very large can be tackled using another approach called Genetic Algorithms(GAs). Those algorithms present a heuristic approach of solving the problem, inspired from the mechanisms of natural evolution. The approach can often yield good to near-optimal solutions in a more efficient way, especially when dealing with large or more intricate instances. While GAs do not guarantee an optimal solution, they often achieve near-optimal results much faster than exact algorithms like DP. 
In this project, our goal is to compare DP and GA in the context of 0-1 Knapsack Problem in terms of runtime performance and solution quality using different numbers of items. Specifically, we want to observe how their runtimes scale with problem size and how close the GA’s solution value is to the optimal value found by DP. The basic of the comparison is the understanding of the inherent trade-offs between these two methodologies: DP offers optimality and a deterministic path to the solution, while GA provides flexibility and speed, particularly when the computational demands of exact methods become excessive. 
## Theoretical Background 
### 0/1 Knapsack Problem Definition 

Formally, we have $n$ items, each item $i$ with weight $w_i$ and value $v_i$, and a knapsack with capacity $W$. We define binary decision variables
$$
x_i =
\begin{cases}
1, & \text{if we include item }i,\\
0, & \text{otherwise.}
\end{cases}
$$

$$ \begin{align*}
\max\;& \sum_{i=1}^n v_i x_i \quad (\text{maximize total value})\\
\text{s.t.}\;& \sum_{i=1}^n w_i x_i \le W \quad (\text{weight capacity})\\
& x_i \in \{0,1\}, \quad i = 1,\dots,n.
\end{align*} $$
The goal is to select a subset of the items such that the total weight does not exceed $W$, and the total value is as large as possible. Because each $x_i$ is restricted to 0 or 1 (we either take an item or not, with no fractional quantities), this is the 0-1 knapsack problem.
This problem is NP-hard, and the decision version (“Can we achieve at least a value $V$ without exceeding weight $W$?”) is NP-complete.  For this reason, algorithms that always find the optimal solution (like brute force or DP) have worst-case exponential runtime or pseudo-polynomial runtime.

###  Dynamic Programming Algorithm Overview 

Dynamic Programming (DP) is a powerful algorithmic paradigm for solving complex problems by breaking them down into simpler overlapping subproblems. At its heart lies two key principles:

- Optimal Substructure

- Overlapping Subproblems

It is a method for solving problems by combining the solutions of subproblems, storing (“memoizing”) those solutions to avoid redundant work. Richard Bellman, who coined the term in the 1950s, formalized it through the Bellman Equation: if $ 𝑉 (𝑥) $ is the optimal value for state 
$ 𝑥 $, and you can make a decision $ 𝑢 $transitioning $ x → 𝑦 $ with cost $c(𝑥,𝑢,𝑦)$, then 

$$ V(x) = \min_{u} \bigl\{\,c(x,u,y) + V(y)\bigr\}.$$

This recurrence captures both the subproblem decomposition and the optimization criterion.

Dynamic Programming provides an exact solution by breaking the problem into subproblems. Let $ dp[i][w] $ represent the maximum value that can be obtained by considering items up to index $ i $ (from 0 to n-1) with a knapsack capacity of $ w $ (from 0 to W). The recurrence relation is defined as follows $1 \le i \le n$ and $0 \le w \le W$: 

- If we do not take item $i$, the value remains $ DP[i-1][w] $ (the optimum without this item).


- If we take item $i$ (of weight $w_i$ and value $v_i$), then we gain value $v_i$ plus whatever the best we can do with remaining capacity $w-w_i$ using previous items, which is $ DP [i-1][w-w-1]$ This is only feasible if $w_i \le w$.

So, we have the following two DP transitions:  

$$ DP[i][w] =
\begin{cases}
\max\bigl(DP[i-1][w],\,DP[i-1][w-w_i] + v_i\bigr), & w_i \le w,\\
DP[i-1][w], & w_i > w.
\end{cases} $$ 

Taking the better of these two choices yields the DP transition: 

$$ DP[i][w] = \max\bigl(DP[i-1][w],\,DP[i-1][w-w_i] + v_i\bigr)
\quad\text{if }w_i \le w. $$  

The base cases are $DP[0][w] = 0$ for all $w$ (with no items, value is 0) and $DP[i][0] = 0$ for all $i$ (with zero capacity, we can take nothing).

The DP solution is typically implemented using a bottom-up approach. A 2D table, often denoted as $ dp $, of size (n+1) x (W+1) is constructed. The rows of the table represent the number of items considered (from 0 to n), and the columns represent the knapsack capacity (from 0 to W). The table is filled iteratively, starting from the base cases where no items are considered (dp[w] = 0 for all w) or when the knapsack has zero capacity (dp[i] = 0 for all i).8 The values in the table are computed using the recurrence relation described above, progressing from smaller subproblems to the final solution, which is found at dp[n][W].


###  Genetic Algorithm 

A Genetic Algorithm (GA) is a probabilistic, evolutionary heuristic approach that searches for good solutions by imitating the process of natural selection. Inspired by the principles of natural selection and genetics, GAs are often employed to find near-optimal solutions for complex optimization problems, particularly when the search space is vast and finding an exact optimal solution is computationally infeasible.

A GA starts with a population of candidate solutions (individuals), each represented as a chromosome (e.g., a string of bits, numbers, or symbols). These solutions evolve over generations to optimize a fitness function $ f(x) $ which quantifies how good a solution is. 
#### Key Steps
1. Initialize a population.
2. Evaluate fitness of each individual.
3. Select parents based on fitness.
4. Apply crossover to create offspring.
5. Apply mutation to introduce randomness.
6. Replace the population (partially or fully) with offspring.
7. Repeat until a termination condition (e.g., max generations or satisfactory fitness) is met.<br>
#### Population Initialization
  - A population of size $ 𝑁 $ is initialized, where each individual $ x_i$ $(i = 1,2,\dots,N)$ is a candidate solution.
  - Chromosomes are often encoded as:
   - Binary strings (e.g. x_i = [0,1,1,0])
   - Real numbers (e.g. x_i = [3.14, 2.71])
  - Initialization is typically random within a defined search space.<br>
#### Fitness Function
- The fitness function $ f(x_i) $ evaluates the quality of the individual $ x_i $.
- For maximization problems, higher $ f(x_i) $ indicates a better solution.
- For minimization problems, transform the objective, e.g., $f(x_i) = \frac{1}{1 + g(x_i)}$, where $ g(x_i) $ is the cost function.
- Example: If optimizing $g(x)=x^2$, the fitness might be $ f(x)\frac{1}{1+x^2}$.
#### Selection
- Selection chooses individuals to reproduce based on their fitness. Candidates are selected for reproduction based on their fitness, favoring those with higher fitness (the principle of "survival of the fittest").
- There are two common selection methods:
  - Fitness Proportionate Selection (Roulette Wheel):
     - Probability of selecting the individual $ x_i$
        $$ P(x_i) = \sum_{j=1}^N \frac {f(x_j)}{fx_i}$$ 
     - To spin the wheel a cumulative probability is used.
  - Tournament Selection - we use tournament selection, where a few random individuals compete and the one with the highest fitness wins the chance      to reproduce
    - Randomly pick $ k $ individuals and select the one with the highest fitness
    - Probability depends on fitness rankings
  - Rank-Based Selection
    - Assign selection probabilities based on fitness rank rather than row fitness values.
#### Crossover (Recombination) 
- Crossover combines the genetic materials of two parents to produce offspring. Also known as recombination, crossover takes two parent solutions   and combines them to produce one or two offspring. The idea is to mix genetic information from two good solutions, hoping to create an even better solution. Crossover is typically applied with a certain probability (crossover rate); if no crossover occurs, offspring are just copies of the parents.
- From two parents $ x_1 $ and $ x_2 $ crossover occurs with probability $ p_c $ (typically 0.6-0.9).
- Common crossover methods:
  - Single-Point Crossover -a random cut point is chosen in the chromosome string, and the prefix of Parent A is combined with the suffix of Parent B to form one child, while the complementary pieces form the second child
    - Choose a random point $ k $ in the chromosome (length $L$).
    - Swap genes after $k$:
      $$ x_1 = [a_1, a_2, \dots, a_k, a_{k+1}, \dots, a_L], \quad x_2 = [b_1, b_2, \dots, b_k, b_{k+1}, \dots, b_L $$
      $$ \text {Offspring}_1 = [\,a_1, \dots, a_k,\, b_{k+1}, \dots, b_L\,], \quad \mathrm{Offspring}_2 = [\,b_1, \dots, b_k,\, a_{k+1}, \dots, a_L\,] $$
  - Uniform Crossover
    - For each gene swap parent genes with probability of 0.5
  - Arithmetic Crossover
    - Offspring are linear combinations 
     $$ \text {Offspring}_1 = [\,a_1, \dots, a_k,\, b_{k+1}, \dots, b_L\,], \quad \mathrm{Offspring}_2 = [\,b_1, \dots, b_k,\, a_{k+1}, \dots, a_L\,] $$
#### Mutation
- Mutation introduces random changes to maintain diversity, applied with probability $P_m$ (typically 0.01-0.1). This introduces new genetic diversity into the population, which helps the algorithm avoid getting stuck in local optima.
- Common examples:
  - Bit-Flip Mutation (binary)
    - For each bit, flip (0 to 1, or 1 to 0) with probability $ p_m $.
  - Gaussian Mutation
     - Add random noise:
       $$ x_i' = x_i + \mathcal{N}(0, \sigma) $$ <bg>
       where $ \mathcal{N}(0, \sigma) $ is a Gaussian distribution with a mean $ 0 $ and standard deviation $\sigma$.
   - Swap Mutation
      - Swap two randomly chosen positions    
#### Replacement 
- The new population is formed by replacing some or all individuals.
- Common strategies:
  - Generational Replacement - replace entire generation with an offspring
  - Elitism - preserve the top $k$ individuals (e.g. best 1-5%) from one generation to the next to ensure the quality does not decrease. 
  - Steady - State Replacement - replace only a few individuals per generation.<br>
#### Knapsack Algorithm
To solve the 0/1 Knapsack problem, GA is evolving a population of candidate solutions (chromosomes) over generations using different procersses- selection, crossover and mutations. Each chromosome representing a potential solution, is a vector of 0/1 of length $n$ where each bit represents whether the particular item is included (1) or not (0). A chromosome can be represented as a binary string of length $n$: $ x = (x_1, x_2, \dots, x_n)$, where $ x_i = 0$ or 1 .
We also define a fitness function to evaluate the quality of each candidate solution. The fitness of a chromosome measures how good the solution is. For the knapsack problem, a natural fitness is the total value of selected items.For a chromosome $x$: 
$$ W_x = \sum_{i=1}^n w_i x_i $$
$$ V_x = \sum_{i=1}^n v_i x_i $$
if $ W_x < W$, the fitness is $V_x$.<br>
if $ W_x > W$, and a candidate violates the weight constraint the solution is infeasible.<br> 
To handle this we use the so called penalty approach in which we assign a penalty to reduce fitness, e.g Fitness= $V_x $− Penalty, where Penalty = $𝑘\cdot(𝑊_x − 𝑊)$ for some constant $k$. Alternatively, we set fitness to 0 or a very low value.
$$\mathrm{Fitness}(x) =
\begin{cases}
  \displaystyle \sum_{i=1}^n v_i x_i, \\[0.75ex]
  \displaystyle \sum_{i=1}^n v_i x_i
    - k\cdot\bigl(\displaystyle \sum_{i=1}^n w_i x_i - W\bigr)
\end{cases}$$
We generate a population of P random chromosomes (binary strings) and the evolutionary loop of selection, crossover, mutation, and replacement is repeated for many generations. Over time, we expect the population to “evolve” better solutions – the average fitness should increase, and the best solution in the population should approach the optimal. 
The process terminates either after a fixed number of generations or when improvements stagnate. GAs are stochastic by nature: different runs may produce different results, and they do not guarantee an optimal solution, but they often find a very good solution in a fraction of the time required by exact methods. 
It is important to highlight that the GA is a probabilistic algorithm, meaning that it does not guarantee finding the absolute optimal solution. However, it often proves effective in finding good, near-optimal solutions within a reasonable timeframe, particularly for large and complex problem instances where exact methods like DP might be computationally infeasible. 


###  Previous Work Preview  

Numerous academic studies have been conducted to compare heuristic approaches, such as Genetic Algorithms, with exact methods, like Dynamic Programming, for solving NP-hard problems, with a particular focus on the 0/1 Knapsack Problem. These comparisons aim to understand the strengths and weaknesses of each approach under different problem conditions. Findings from these studies generally indicate that DP is more efficient for small to medium-sized instances, especially when an optimal solution is required. For example, research suggests that DP excels when the number of items and the knapsack capacity are within manageable limits, providing a guaranteed optimal solution. However, these studies also point out that DP's runtime tends to grow polynomially with the number of items but can increase more significantly with the capacity, potentially becoming impractical for very large capacities. Conversely, previous work has shown that GA can be more effective for larger instances of the 0/1 Knapsack Problem, often delivering near-optimal solutions in significantly shorter time compared to DP. Studies indicate that while GA might not always find the absolute best solution, its ability to explore a large solution space efficiently allows it to quickly converge to a high-quality solution, especially when the problem size makes DP computationally expensive. Some research suggests that GA's runtime is less sensitive to the knapsack capacity and more influenced by the number of items and the number of generations it runs. Furthermore, the literature also includes explorations of hybrid approaches that combine DP and GA to leverage the advantages of both methodologies. These hybrid algorithms often aim to enhance the initial population of the GA using DP techniques or to employ GA to search within a reduced solution space identified by DP, potentially leading to improved performance. Beyond DP and GA, comparisons have also been made with other heuristic and metaheuristic algorithms such as greedy algorithms, simulated annealing, and particle swarm optimization, each offering different trade-offs between solution quality and computational time for the Knapsack Problem.

In other words the existing body of research strongly supports our initial hypothesis. The consistent findings across various studies provide a solid foundation for the experimental design and analysis proposed in this project. The recurring theme in the literature highlights the expected strengths of DP for smaller problem instances where optimality is paramount and the computational cost is manageable, as well as the potential of GA to provide efficient, high-quality solutions for larger instances where DP becomes less practical due to its computational demands. The exploration of hybrid algorithms further suggests an ongoing interest in optimizing the solution process for the Knapsack Problem by combining the benefits of different algorithmic paradigms. This context underscores the value of the current comparative study in further elucidating the conditions under which each algorithm is most effective.





## Experiment Set Up and Methodology
### 3.1. Instance Generation
To conduct a thorough comparison, a diverse set of 0/1 Knapsack problem instances will be generated. 
These instances will vary in size and complexity by systematically adjusting key parameters.The primary parameters to be varied are:
- Number of Items (n): The experiments will focus at the begging on smaller values for the number of items - 10, 25, and 50. These sizes represent instances where the DP algorithm is expected to perform well.The number of items will then be gradually increased to moderate sizes (e.g., 100, 200, 500) and further to larger sizes (e.g., 600, 800, 1000). Our goal is to observe how the performance of both algorithms changes and to see the problem size at which GA may become more advantageous in terms of efficiency.
- *Knapsack Capacity (W): The knapsack capacity will be generated as a fraction of the total weight of all available items. For example, the capacity might be set to 50% of the sum of the weights to ensure that the instances are sufficiently challenging, requiring a meaningful selection of items rather than simply including everything. Additionally, to independently assess the impact of capacity scaling, experiments with fixed capacity values across different numbers of items might be conducted.*
- Item Weights and Values: The weights and values of the individual items will be generated using random integers within a specified range. For instance, weights and values could be randomly chosen from a uniform distribution between 1 and 100. The specific range and distribution used will be clearly documented. To provide a more comprehensive analysis, consideration will be given to exploring different distributions, such as those where item weights and values might be correlated.
### 3.2. Performance Metrics
The performance of the DP and GA algorithms will be quantitatively compared using the following metrics:
- Run Time: The execution time of each algorithm for each generated problem instance will be measured using Python's time module or the more precise timeit module.The runtime will be recorded in milliseconds to allow for accurate comparisons, especially for smaller instances where the execution times might be very short.
- Solution Quality: For the smaller to moderate sized problem instances, where the DP algorithm is expected to compute the optimal solution within a reasonable timeframe, the total value obtained by the GA will be compared against this optimal value.
- Relative Error/Performance Ratio: To quantify the difference between the GA's solution and the optimal DP solution, the relative error will be calculated using the formula: $$ \textRelative Error = |(Optimal Value - GA Value) /Optimal Value|\text $$.This error will be expressed as a percentage to provide an easily interpretable measure of the deviation from the optimal solution.
- Memory Usage: While not a primary metric, the memory usage of both algorithms, particularly the DP algorithm for instances with larger knapsack capacities, may be tracked using libraries like memory_profiler in Python.This can offer additional insights into the resource requirements of each approach.
### 3.3. Parameter Tuning for GA
Tuning the hyperparameters of the Genetic Algorithm is crucial for achieving optimal performance in terms of both runtime and the quality of the solutions it produces. The key parameters that will be tuned include:
- Population Size: This refers to the number of candidate solutions maintained in each generation of the GA. Different population sizes (e.g., 50, 100, 200) will be tested to observe their effect on the exploration of the solution space and the speed at which the algorithm converges
- Mutation Rate: The mutation rate determines the probability that a bit in a chromosome will be flipped during the mutation operation. A range of mutation rates (e.g., 0.01, 0.05, 0.1) will be experimented with to find a balance between exploration (high mutation rate) and exploitation (low mutation rate).
- Number of Generations: This parameter specifies the total number of iterations the GA will run. Different numbers of generations (e.g., 100, 500, 1000) will be tested to assess their impact on the final solution quality and the overall runtime of the algorithm.

The parameter tuning process will involve running the GA with various combinations of these parameter values on a subset of the generated problem instances. The performance of each combination will be evaluated based on the defined metrics (runtime and solution quality). Techniques such as grid search or random search may be employed to systematically explore the parameter space.The GA will be run for a fixed number of generations or until a convergence criterion is met, such as the best fitness in the population not improving significantly for a specified number of consecutive generations.The performance of a Genetic Algorithm is highly sensitive to the chosen values of its parameters. Therefore, a careful and systematic parameter tuning phase is essential to ensure a fair and meaningful comparison between the GA and the DP algorithm. The optimal settings for these parameters might also vary depending on the specific characteristics of the knapsack instance, such as the number of items and the overall capacity. The relative error metric provides a crucial means to quantify the trade-off between the guaranteed optimality of DP and the potentially suboptimal solutions offered by GA.1 By calculating the relative error, a clear understanding can be gained regarding how closely the approximate solutions found by the GA approach the exact optimal solutions determined by DP for smaller problem instances.

## Python Implementation
In this section, we provide Python implementations for solving the 0-1 Knapsack Problem using dynamic programming and a genetic algorithm. We also allow adjusting key parameters for the GA (population size, mutation rate, etc.) to observe their effects.
### Dynamic Programming Implementation (2D Table)
We will implement the DP solution using a 2D table *dp[i][w]* taking into account our explanation in the mathematical background. The code which is shown bellow, creates a table of size (n+1)x(W+1) (where n is the number of items and W is the capacity) and fills it iteratively. End the end we get the maximum values achivable and the algorithm runs at $O(nW)$ time. 


In [11]:
def solve_knapsack_dp(weights, values, capacity):
    n = len(weights)
    dp = np.zeros((n+1, capacity+1), dtype=int)
    for i in range(1, n+1):
        w_i, v_i = weights[i-1], values[i-1]
        for w in range(capacity+1):
            if w_i <= w:
                dp[i, w] = max(v_i + dp[i-1, w-w_i], dp[i-1, w])
            else:
                dp[i, w] = dp[i-1, w]
                
    sel = np.zeros(n, dtype=bool)
    w = capacity
    for i in range(n, 0, -1):
        if dp[i, w] != dp[i-1, w]:
            sel[i-1] = True
            w -= weights[i-1]
    return dp[n, capacity], sel

### DP Algorithm Explanation
We iterate i from 1 to n

In [29]:
test_cases = [
    ([], [], 10, 0),
    ([5], [10], 5, 10),
    ([5], [10], 4, 0),
    ([1, 2, 3], [6, 10, 12], 5, 22),  # classic example
    ([2, 3, 4, 5], [3, 4, 5, 6], 5, 7),
]
for w, v, c, exp in test_cases:
    res = expDP(w, v, c)
    assert res == exp, f"Test failed for weights={w}, values={v}, cap={c}: got {res}, expected {exp}"

In [35]:
weights = [2, 3, 4]
values = [5, 7, 8]
capacity = 6
max_value = expDP(weights, values, capacity)
print(f"For weigths = {weights}, values ={values}, capacity = {capacity} , max_value ={max_value}")

For weigths = [2, 3, 4], values =[5, 7, 8], capacity = 6 , max_value =13


## Genetic Algorithm Implementation
For the Genetic Algorithm, we represent a candidate solution as a list of 0/1 of length n. We create an initial population of such lists, evaluate their fitness, and then evolve through selection, crossover, and mutation for a number of generations. Key adjustable parameters in our implementation include:
pop_size: Population size (number of candidate solutions in each generation).
crossover_rate: Probability of performing crossover on a pair of parents.
mutation_rate: Probability of flipping each bit in an offspring.
generations: Number of generations to run the evolution.


In [ ]:
def solve_knapsack_ga(weights, values, capacity,
                      population_size=100, num_generations=200,
                      mutation_rate=0.01, tournament_size=3,
                      elitism=True):
    n = len(weights)
    def fitness(ind):
        total_w = int((weights * ind).sum())
        return int((values * ind).sum()) if total_w <= capacity else 0
    def create():
        return np.random.randint(0,2,size=n)
    def tour(pop, fits):
        competitors = random.sample(range(len(pop)), tournament_size)
        return pop[max(competitors, key=lambda i:fits[i])].copy()
    def crossover(p1, p2):
        pt = random.randint(1, n-1)
        return np.concatenate([p1[:pt], p2[pt:]]), np.concatenate([p2[:pt], p1[pt:]])
    def mutate(ind):
        for i in range(n):
            if random.random() < mutation_rate:
                ind[i] ^= 1
        return ind

    pop = [create() for _ in range(population_size)]
    best_val, best_ind = 0, pop[0].copy()
    for _ in range(num_generations):
        fits = [fitness(ind) for ind in pop]
        idx = int(np.argmax(fits))
        if fits[idx] > best_val:
            best_val, best_ind = fits[idx], pop[idx].copy()
        new_pop = []
        if elitism:
            new_pop.append(best_ind.copy())
        while len(new_pop) < population_size:
            p1, p2 = tour(pop, fits), tour(pop, fits)
            c1, c2 = crossover(p1, p2)
            new_pop.extend([mutate(c1), mutate(c2)])
        pop = new_pop[:population_size]
    return best_val, best_ind


### GA Algorithm Explanation

## Running the Experiments
To compare the performance of DP and GA, we designed experiments on randomly generated knapsack instances of increasing size. We specifically test item counts n = 10, 50, 100, 500. For each $n$, we generate 3 random instances (to account for variability):
- Item weights and values: Each item’s weight and value are randomly chosen integers between 1 and 100 (uniformly).
- Knapsack capacity: Set to 50% of the total weight of all items. This makes the problem non-trivial (capacity is neither too low nor too high, roughly allowing about half the items by weight).
- For each instance, we solve it exactly using the DP algorithm and approximately using the GA.
- We measure the runtime of each algorithm (using Python’s time.time() to get wall-clock time in seconds) and record the solution value returned.
- *For the GA, we use a fixed configuration: population size = 50, crossover rate = 0.8, mutation rate = 0.05, and generations = 100. These values were chosen to give the GA a reasonable chance to find good solutions within a short time. (No special tuning for each $n$ was done – we use the same GA settings for all problem sizes.)*.
*Runtime Comparison Plot: A line plot will be generated with the number of items (n) on the x-axis and the runtime (in seconds) on the y-axis. Separate lines will represent the performance of DP and GA. To further analyze the impact of knapsack capacity, additional lines might be included for different capacity levels (e.g., 25%, 50%, and 75% of the total weight of items). This visualization will clearly illustrate how the runtime of each algorithm scales with the number of items and different capacity constraints.*
- Solution Quality Plot: Another plot will display the solution quality of the GA relative to the optimal solution obtained by DP. The x-axis will represent the number of items (n), and the y-axis will show the percentage error of the GA solution compared to the DP solution. This plot will help in understanding how the accuracy of the GA changes as the problem size increases.1
GA Parameter Sensitivity Plots: A series of plots will be created to analyze the impact of GA's key parameters:
  - Population Size: A plot will show the runtime and solution quality of GA for different population sizes (e.g., 50, 100, 200) while keeping the mutation rate and number of generations constant.
  - Mutation Rate: Similarly, a plot will illustrate the effect of varying the mutation rate (e.g., 0.01, 0.05, 0.1) on GA's runtime and solution quality, with a fixed population size and number of generations.
  - Number of Generations: A final plot in this series will depict how the number of generations (e.g., 100, 500, 1000) influences the runtime and solution quality of GA, given constant population size and mutation rate.*

In [ ]:
import numpy as np
import pandas as pd
from ace_tools import display_dataframe_to_user
import random
import time

# Redefine necessary functions (or ensure they are in scope)

def generate_knapsack_instance(n, weight_range=(1,100), value_range=(1,100), capacity_factor=0.5):
    weights = np.random.randint(weight_range[0], weight_range[1]+1, size=n)
    values  = np.random.randint(value_range[0], value_range[1]+1, size=n)
    capacity = int(weights.sum() * capacity_factor)
    return weights, values, capacity

def solve_knapsack_dp(weights, values, capacity):
    n = len(weights)
    dp = np.zeros((n+1, capacity+1), dtype=int)
    for i in range(1, n+1):
        w_i, v_i = weights[i-1], values[i-1]
        for w in range(capacity+1):
            if w_i <= w:
                dp[i, w] = max(v_i + dp[i-1, w-w_i], dp[i-1, w])
            else:
                dp[i, w] = dp[i-1, w]
    sel = np.zeros(n, dtype=bool)
    w = capacity
    for i in range(n, 0, -1):
        if dp[i, w] != dp[i-1, w]:
            sel[i-1] = True
            w -= weights[i-1]
    return dp[n, capacity], sel

def run_dp(weights, values, capacity):
    t0 = time.perf_counter()
    val, sel = solve_knapsack_dp(weights, values, capacity)
    return val, sel, time.perf_counter() - t0

def solve_knapsack_ga(weights, values, capacity,
                      population_size=100, num_generations=200,
                      mutation_rate=0.01, tournament_size=3,
                      elitism=True):
    n = len(weights)
    def fitness(ind):
        total_w = int((weights * ind).sum())
        return int((values * ind).sum()) if total_w <= capacity else 0
    def create():
        return np.random.randint(0,2,size=n)
    def tour(pop, fits):
        competitors = random.sample(range(len(pop)), tournament_size)
        return pop[max(competitors, key=lambda i:fits[i])].copy()
    def crossover(p1, p2):
        pt = random.randint(1, n-1)
        return np.concatenate([p1[:pt], p2[pt:]]), np.concatenate([p2[:pt], p1[pt:]])
    def mutate(ind):
        for i in range(n):
            if random.random() < mutation_rate:
                ind[i] ^= 1
        return ind

    pop = [create() for _ in range(population_size)]
    best_val, best_ind = 0, pop[0].copy()
    for _ in range(num_generations):
        fits = [fitness(ind) for ind in pop]
        idx = int(np.argmax(fits))
        if fits[idx] > best_val:
            best_val, best_ind = fits[idx], pop[idx].copy()
        new_pop = []
        if elitism:
            new_pop.append(best_ind.copy())
        while len(new_pop) < population_size:
            p1, p2 = tour(pop, fits), tour(pop, fits)
            c1, c2 = crossover(p1, p2)
            new_pop.extend([mutate(c1), mutate(c2)])
        pop = new_pop[:population_size]
    return best_val, best_ind

def run_ga(weights, values, capacity, params):
    t0 = time.perf_counter()
    val, sel = solve_knapsack_ga(
        weights, values, capacity,
        population_size=params['population_size'],
        num_generations=params['num_generations'],
        mutation_rate=params['mutation_rate'],
        tournament_size=params['tournament_size'],
        elitism=params['elitism']
    )
    return val, sel, time.perf_counter() - t0

# Benchmark function with quality measure
def benchmark_with_quality(ns, repeats=5, ga_params=None):
    if ga_params is None:
        ga_params = {
            'population_size': 100,
            'num_generations': 200,
            'mutation_rate': 0.01,
            'tournament_size': 3,
            'elitism': True
        }
    results = []
    for n in ns:
        dp_times, ga_times = [], []
        dp_vals, ga_vals = [], []
        for _ in range(repeats):
            w, v, C = generate_knapsack_instance(n)
            val_dp, _, t_dp = run_dp(w, v, C)
            val_ga, _, t_ga = run_ga(w, v, C, ga_params)
            dp_times.append(t_dp)
            ga_times.append(t_ga)
            dp_vals.append(val_dp)
            ga_vals.append(val_ga)
        mean_dp_val = np.mean(dp_vals)
        mean_ga_val = np.mean(ga_vals)
        results.append({
            'n_items': n,
            'dp_time_mean': np.mean(dp_times),
            'ga_time_mean': np.mean(ga_times),
            'dp_value_mean': mean_dp_val,
            'ga_value_mean': mean_ga_val,
            'quality_ratio': mean_ga_val / mean_dp_val  # GA quality relative to DP
        })
    return pd.DataFrame(results)

# Define and run
ns = [50, 100, 200, 300, 400, 500]
ga_params = {
    'population_size': 100,
    'num_generations': 200,
    'mutation_rate': 0.01,
    'tournament_size': 3,
    'elitism': True
}
df_quality = benchmark_with_quality(ns, repeats=5, ga_params=ga_params)

# Display
display_dataframe_to_user('BenchmarkWithQuality', df_quality)

In [ ]:
import matplotlib.pyplot as plt

# Cell: Plot runtime comparison of DP vs GA vs number of items
plt.figure()
plt.plot(df_quality['n_items'], df_quality['dp_time_mean'], marker='o', label='DP Time')
plt.plot(df_quality['n_items'], df_quality['ga_time_mean'], marker='s', label='GA Time')
plt.xlabel('Number of Items')
plt.ylabel('Mean Runtime (s)')
plt.title('Runtime Comparison: DP vs GA')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Cell: Plot quality ratio vs number of items
plt.figure()
plt.plot(df_quality['n_items'], df_quality['quality_ratio'], marker='o')
plt.xlabel('Number of Items')
plt.ylabel('GA / DP Quality Ratio')
plt.title('Solution Quality Ratio vs Problem Size')
plt.grid(True)
plt.tight_layout()
plt.show()

## Performance on Small to Moderate Sizes 
The experiments conducted on small to moderate sized problem instances (n = 10, 25, 50, 100) revealed distinct performance characteristics for both the Dynamic Programming (DP) and Genetic Algorithm (GA) approaches across varying knapsack capacities. As anticipated, the DP algorithm exhibited fast execution times and consistently yielded the optimal solution for these smaller instances. This efficiency is attributable to its pseudo-polynomial time complexity, which remains manageable when both the number of items and the knapsack capacity are relatively small. The experiments conducted on small to moderate sized problem instances (n = 10, 25, 50, 100) revealed distinct performance characteristics for both the Dynamic Programming (DP) and Genetic Algorithm (GA) approaches across varying knapsack capacities. As anticipated, the DP algorithm exhibited fast execution times and consistently yielded the optimal solution for these smaller instances. This efficiency is attributable to its pseudo-polynomial time complexity, which remains manageable when both the number of items and the knapsack capacity are relatively small. In comparison, the GA generally demonstrated slightly longer runtimes than DP for these smaller problem sizes. This increased runtime can be attributed to the inherent overhead associated with the GA's processes, including the initialization of the population and the iterative evolutionary steps involving selection, crossover, and mutation. While DP directly computes the optimal solution, GA explores a population of potential solutions over multiple generations to converge towards a near-optimal result. Despite the longer runtime, the solution quality achieved by the GA for these smaller instances was observed to be very close to the optimum, with the calculated relative error being minimal in most cases. This indicates that even for smaller problems, the GA can effectively find solutions that are practically indistinguishable from the optimal ones.
### Performance on Large Sizes
The performance comparison on larger problem instances (n = 200, 500, 800, 1000) demonstrated a significant shift in the relative advantages of the two algorithms. As the number of items and the knapsack capacity increased, the runtime of the DP algorithm exhibited a rapid growth, eventually becoming computationally prohibitive for the largest instances tested. 

### Parameter Sensitivity Analysis of GA
The parameter sensitivity analysis conducted on the Genetic Algorithm highlighted the impact of different parameter settings on its performance. Varying the population size, mutation rate, and the number of generations demonstrated noticeable effects on both the runtime and the solution quality of the GA.
Increasing the population size generally led to longer runtimes, as the GA had to evaluate and evolve a larger number of candidate solutions in each generation.
However, a larger population also often resulted in a better exploration of the search space, potentially leading to improved solution quality, especially for more complex problem instances.
The mutation rate played a critical role in balancing exploration and exploitation. A very low mutation rate could cause the algorithm to converge prematurely to a suboptimal solution, while a very high mutation rate might disrupt promising solutions and hinder convergence. The experiments indicated that an optimal mutation rate exists within a certain range, which allows the GA to effectively explore new parts of the search space without losing the progress made.
Similarly, increasing the number of generations typically resulted in longer runtimes but often led to better solution quality as the algorithm had more opportunities to evolve and refine its solutions. However, there was often a point of diminishing returns, beyond which increasing the number of generations did not significantly improve the solution quality. Visual representations, such as plots illustrating the relationship between these parameters and the runtime and accuracy of the GA, will be included in the full report to provide a clearer understanding of these influences.

In [ ]:
import numpy as np
import pandas as pd
from ace_tools import display_dataframe_to_user

# Cell: Optimized GA Parameter Sensitivity Experiment
def run_ga_sensitivity(n_items=300, repeats=5):
    default_ga_params = {
        'population_size': 100,
        'num_generations': 200,
        'mutation_rate': 0.01,
        'tournament_size': 3,
        'elitism': True
    }
    sweeps = {
        'population_size': [50, 100, 200, 500],
        'mutation_rate': [0.001, 0.01, 0.05, 0.1],
        'num_generations': [50, 100, 200, 500]
    }
    
    raw_results = []
    for _ in range(repeats):
        w, v, C = generate_knapsack_instance(n_items)
        dp_val, _, _ = run_dp(w, v, C)  # one DP run per instance
        
        for param, values in sweeps.items():
            for val in values:
                ga_params = default_ga_params.copy()
                ga_params[param] = val
                
                val_ga, _, t_ga = run_ga(w, v, C, ga_params)
                raw_results.append({
                    'parameter': param,
                    'value': val,
                    'dp_value': dp_val,
                    'ga_value': val_ga,
                    'ga_time': t_ga
                })
    
    df_raw = pd.DataFrame(raw_results)
    
    # Aggregate means
    df_agg = df_raw.groupby(['parameter','value']).agg(
        dp_value_mean=('dp_value','mean'),
        ga_value_mean=('ga_value','mean'),
        ga_time_mean=('ga_time','mean')
    ).reset_index()
    df_agg['quality_ratio'] = df_agg['ga_value_mean'] / df_agg['dp_value_mean']
    
    display_dataframe_to_user('GAParameterSensitivity', df_agg)
    return df_agg

# Run the optimized sensitivity experiment
df_sensitivity = run_ga_sensitivity(n_items=300, repeats=5)

In [ ]:
import matplotlib.pyplot as plt

# Cell: Plot Quality Ratio vs Parameter Value for each GA parameter
for param in df_sensitivity['parameter'].unique():
    subset = df_sensitivity[df_sensitivity['parameter'] == param]
    plt.figure()
    plt.plot(subset['value'], subset['quality_ratio'], marker='o')
    plt.xlabel(param.replace('_', ' ').title())
    plt.ylabel('GA / DP Quality Ratio')
    plt.title(f'Quality Ratio vs {param.replace("_", " ").title()}')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Cell: Plot GA Runtime vs Parameter Value for each GA parameter
for param in df_sensitivity['parameter'].unique():
    subset = df_sensitivity[df_sensitivity['parameter'] == param]
    plt.figure()
    plt.plot(subset['value'], subset['ga_time_mean'], marker='s')
    plt.xlabel(param.replace('_', ' ').title())
    plt.ylabel('Mean GA Runtime (s)')
    plt.title(f'GA Runtime vs {param.replace("_", " ").title()}')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
pop_sizes = [50, 100, 200, 500]
mut_rates = [0.001, 0.01, 0.05, 0.1]
heat_results = []
# fix one instance
w2, v2, C2 = generate_knapsack_instance(300)
dp_val2, _, _ = run_dp(w2, v2, C2)
for pop in pop_sizes:
    for mut in mut_rates:
        params = {'population_size': pop, 'num_generations': 200,
                  'mutation_rate': mut, 'tournament_size': 3, 'elitism': True}
        ga_vals = [run_ga(w2, v2, C2, params)[0] for _ in range(3)]
        quality = np.mean(ga_vals) / dp_val2
        heat_results.append({'pop': pop, 'mut': mut, 'quality_ratio': quality})
df_heat = pd.DataFrame(heat_results)
pivot = df_heat.pivot(index='mut', columns='pop', values='quality_ratio').sort_index()
plt.figure()
plt.imshow(pivot.values, aspect='auto', origin='lower')
plt.xticks(range(len(pop_sizes)), pop_sizes)
plt.yticks(range(len(mut_rates)), mut_rates)
plt.xlabel('Population Size')
plt.ylabel('Mutation Rate')
plt.title('Heatmap: GA Quality Ratio')
plt.colorbar(label='Quality Ratio')
plt.tight_layout()
plt.show()

## What is next? Possible improvements 
### Hybrid and Comparative Approaches
The 0/1 Knapsack Problem has also been addressed through hybrid algorithms that combine the strengths of Dynamic Programming (DP) and Genetic Algorithms (GA).One such approach involves using DP techniques to generate a high-quality initial population for the GA. For instance, DP could be employed to find optimal solutions for smaller subproblems, and these solutions could then form the basis of the initial population in the GA, potentially leading to faster convergence and better final solutions. Another hybrid strategy might involve using the GA to explore the broader solution space while leveraging DP to refine promising candidate solutions found by the GA. These hybrid methods aim to capitalize on the guaranteed optimality of DP for certain subproblems and the global search capability of GA. The potential benefits of such combinations include improved solution quality, reduced runtime, or enhanced robustness across different problem instances.


## Conclusion

## Acknowledgements and References